In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img

from PIL import Image
import matplotlib.pyplot as plt

import cv2

## Step 1、2、3

In [ ]:
# 設定文件夾路徑和圖像大小
X_path = "C://ETT_v3/Fold1/train/"
y_path = "C://ETT_v3/Fold1/trainannot/"
img_size = (256, 256)

# 取得文件夾中所有的圖像檔名
X_names = os.listdir(X_path)
y_names = os.listdir(y_path)

# 初始化空的numpy陣列以儲存圖像
X_data = np.zeros((len(X_names)*2, img_size[0], img_size[1], 3))
y_data = np.zeros((len(X_names)*2, img_size[0], img_size[1], 3))

# 遍歷每個圖檔，使用Load_img()讀取圖檔
for i, X_name in enumerate(X_names):
    img = load_img(X_path + X_name, target_size=img_size)
    img_arr = np.array(img)
    img_gray = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
    
    # 對比度處理 
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_clahe = clahe.apply(img_gray)
    img_clahe_color = cv2.cvtColor(img_clahe, cv2.COLOR_GRAY2RGB)
    X_data[i] = img_clahe_color
    # 水平翻轉處理
    img_flipped = cv2.flip(img_clahe_color, 1)
    X_data[i+len(X_names)] = img_flipped
    
    # 顯示原始圖像、CLAHE處理後和水平翻轉後的圖像
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title('Original Image')
    
    plt.subplot(1, 3, 2)
    plt.imshow(img_clahe_color)
    plt.title('CLAHE Image')
    
    plt.subplot(1, 3, 3)
    plt.imshow(img_flipped)
    plt.title('Flipped Image')
    plt.show()

# 列出image陣列維度
print("Image data shape:", X_data.shape)

In [ ]:
# 遍歷每個mask圖檔，使用Load_img()讀取圖檔
for i, y_name in enumerate(y_names):
    mask = load_img(y_path + y_name, target_size=img_size)
    mask_arr = np.array(mask)
    
    # 水平翻轉處理
    mask_flipped = cv2.flip(mask_arr, 1)
    y_data[i] = mask_arr
    y_data[i+len(y_names)] = mask_flipped
    
    # 顯示原始mask、水平翻轉後的mask
    plt.subplot(1, 2, 1)
    plt.imshow(mask_arr)
    plt.title('Original Mask')
    plt.subplot(1, 2, 2)
    plt.imshow(mask_flipped)
    plt.title('Flipped Mask')
    plt.show()

# 列出mask陣列維度
print("Image data shape:", y_data.shape)

## Step 4

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

# 超參數
img_size = (160, 160)
num_classes = 4
batch_size = 32

def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model


# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
model = get_model(img_size, num_classes)
model.summary()